# Fraud Detection Hands-on Example

## Simple Fraud Model: 
* Track all customer accounts.
* For any specific customer account:
 *  Maintain average transaction amount for each payee.
 *  Flag transactions into a given account that appear fraudulent.
 *  Have the ability to tune the fraud detection criteria.

<img src="./graphics/fraud_detection_1.png"
     alt="Fraud Detection"
     style="center; margin-right: 10px;"
     width="800"
     height="640"/>

## Presentation Outline 
* Quick review of the Jupyter Notebook Environment
* Interactive Java Development Environment
* Connecting to Aerospike One node cluster 
* Simple Fraud Detection Problem Description
* Solution Development



### Jupyter Notebook Overview
What is happening behind the scenes...

![fig2](./graphics/JupNb_iPython_Overview.png)

We are using the iPython (Interactive Python Kernel for Jupyter Notebook) in this example.

We also have the option to code in Java - in a separate Notebook, iJava is the other interactive Kernel available in Jupyter Notebooks.

##### Jupyter Notebook - Cells and Execution / Edit shortcuts 

We have a mix of Markdown cells (text description in Markdown) and Code cells (we have chosen iPython Kernel).
<img src="./graphics/JupNb_iPython_Cells.png"
     alt="Fraud Detection"
     style="center; margin-right: 10px;"
     width="250"
     height="400"/>


You can use the menu above to change a cell from being a Code cell to a Markdown cell.

**Cntrl-Enter** executes a cell. Try it.

Clicking anywhere in text display (Markdown) cell and hitting **Enter**, puts you in the cell edit mode. Try it.

Code cells can be edited directly. To run an edited code cell, just hit **Cntrl-Enter**.

You can add a new blank cell, cells can be deleted, copied, pasted or moved up or down using the menu icons as shown below:

![fig4](./graphics/JupNb_CellEditing.png)

### Housekeeping - Wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _aql_.

This is needed while doing code development. If you want to clear the iPython Kernel of all Python objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

Let us run some basic aql commands from a cell that _"becomes a shell"_ (using %%sh directive). 

We insert a record, read all records in namespace test, truncate namespace test, again read all records in namespace test - we expect there should be _no records in namespace test after truncate_. 

In [1]:
%%sh
aql 
insert into test.testset (pk, b1) values (1,1)
select * from test
truncate test
select * from test
exit

Seed:         127.0.0.1
User:         None
Config File:  /etc/aerospike/astools.conf /home/training/.aerospike/astools.conf 
Aerospike Query Client
Version dev
Copyright 2012-2021 Aerospike. All rights reserved.
aql> insert into test.testset (pk, b1) values (1,1)
OK, 1 record affected.

aql> select * from test
+-------------+----------+--------+-------+------------+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| ovr         | lastamnt | payee  | bal   | profile    | thresh | mapPayees                                                                                                                                                                                                                                                      |
+-------------+----------+--------+-------

C Client Version 6.0.0


### Runing aql via Terminal Tab

Alternatively, you can also open a separate terminal window (TAB) from the Home Page and run aql interactively in the Terminal.  Please **DO NOT CLICK ON QUIT** on Login Page. It will kill the Jupyter Notebook Server.

![fig5](./graphics/JupNb_Terminal.png)

### Get our Aerospike Python Client talking to the Single Node Aerospike Server 

We will be discussing the data model development interactively. We will code as we progress through the discussion. So let us connect our client application's interactive coding environment with the single node Aerospike server. This will get our real time interaction started.

As we write code in following Jupyter Notebook cells, it will be executed on the Single node Aerospike Server. 

#### Add Python Client required imports


In [2]:
import aerospike
print("Client module imported.")

Client module imported.


#### Connect to the Aerospike Server
Create the config dictionary with the seed host.
Using the config dictionary, create the client object.
Instantiate the client object. Let us write a record and read it back.
We have a namespace **_test_** pre-defined on the server.

In [3]:
config = { 'hosts':[('127.0.0.1',3000)]}
client = aerospike.client(config)
try:
    client.connect()
except:
    import sys
    print("Failed to connect to the cluster with", config['hosts'])
    sys.exit(1)
print("Connected to the cluster")

Connected to the cluster


#### Test Record Data Model

![fig_DM_1](./graphics/Fraud_DM_1.png)

In [4]:
#Insert a record with two bins, b1 with string v1 and b2 with integer 2 as data.
#Bins are specified as a dictionary in the put() API.

mykey = ('test', 'testset', 'key1')  #Define the key tuple (namespace, set, key)
print("We will insert a record with mykey =", mykey)

bins = {'b1':'val1', 'b2':2}

meta = {}  # e.g. {'ttl':0}

wPolicy = {'key': aerospike.POLICY_KEY_SEND}

try:
    client.put(mykey, bins, policy=wPolicy)  # put(key, bins, meta, policy)
except Exception as e:
    import sys
    print("error: {0}".format(e), file=sys.stderr)
    sys.exit(1)
print('Successfully written the record. Lets check with aql.')


We will insert a record with mykey = ('test', 'testset', 'key1')
Successfully written the record. Lets check with aql.


In [5]:
%%sh
aql 
select * from test
exit

Seed:         127.0.0.1
User:         None
Config File:  /etc/aerospike/astools.conf /home/training/.aerospike/astools.conf 
Aerospike Query Client
Version dev
Copyright 2012-2021 Aerospike. All rights reserved.
aql> select * from test
+--------+--------+----+
| PK     | b1     | b2 |
+--------+--------+----+
| "key1" | "val1" | 2  |
+--------+--------+----+
1 row in set (0.042 secs)

OK

aql> exit



C Client Version 6.0.0


##### About Jupyter Notebook Cell Execution sequence

If a previous code cell was executed, objects instantiated in it are available in the next cell that you execute. 

You can execute individual cells in any order - this is interactive execution.

Order of execution will determine the net result.  i.e. you can go back and forth between the record insertion cell above, and record read cell below. 

![fig_exec](./graphics/JupNb_iPython_Execution.png)

If you edit the insertion values, you must re-execute the write cell to put the new value on the Aerospike Server. Then, if you execute the read cell, you will see the updated value.

**_If you have lost track about where you are in the sequence, go to the top Menu, under Cell, Choose "Run All" to run all cells - top to bottom - in this notebook's sequence._** If you want to clear all objects in the kernel and restart from a fresh state, use **Kernel -> Restart & Run All**.


In [6]:
#Read the record that we just inserted

(key, metadata, bins) = client.get(mykey, policy={'key': aerospike.POLICY_KEY_SEND})  #pass policy object to read stored key

#(key, metadata, bins) = client.get(mykey, policy={})

print("Read back the record.")
print("Record's key is:", key)
print("Record Metadata items are:", metadata)
print("Record Bin Values are:", bins)


Read back the record.
Record's key is: ('test', 'testset', 'key1', bytearray(b"\xbfl\x1d\x13\xe7\xcd\x10\xc5\xbd\x02-\'\xe7\xdf\x17\x0c\x0b\xcc\xd6\xe1"))
Record Metadata items are: {'ttl': 2592000, 'gen': 1}
Record Bin Values are: {'b1': 'val1', 'b2': 2}


In [7]:
digest = b"\xbfl\x1d\x13\xe7\xcd\x10\xc5\xbd\x02-\'\xe7\xdf\x17\x0c\x0b\xcc\xd6\xe1"
print(bytearray(digest).hex().upper())

BF6C1D13E7CD10C5BD022D27E7DF170C0BCCD6E1


In [8]:
%%sh
aql 
set output json
explain select * from test.testset where pk="key1"
exit

Seed:         127.0.0.1
User:         None
Config File:  /etc/aerospike/astools.conf /home/training/.aerospike/astools.conf 
Aerospike Query Client
Version dev
Copyright 2012-2021 Aerospike. All rights reserved.
aql> set output json
OUTPUT = JSON
aql> explain select * from test.testset where pk="key1"

[
    [
        {
          "TIMEOUT": 1000,
          "UDF": "FALSE",
          "MASTER NODE": "BB9633A29C2EA02",
          "SET": "testset",
          "NAMESPACE": "test",
          "DIGEST": "BF6C1D13E7CD10C5BD022D27E7DF170C0BCCD6E1",
          "PARTITION": 3263,
          "STATUS": "",
          "KEY_TYPE": "STRING",
          "POLICY_KEY": "AS_POLICY_KEY_SEND"
        }
    ],
    [
        {
          "Status": 0
        }
    ]
]

aql> exit



C Client Version 6.0.0


## Getting Back to Fraud Detection
Now that we are all setup, let us get back to discussing our Fraud Detection Example.

## Simple Incoming Transaction

* New Account:

 *  acct_id:  _**override**_, _**withdrawl_amount**_, _**payee**_, current_balance, user_profile_json 
 
* New Transaction to existing account:

 *  acct_id:  _**override**_, _**withdrawl_amount**_, _**payee**_ 


We can use the DocumentAPI to write to the user_profile_json bin. 

However, for this hands-on, we will focus on the fraud detection part. For simplicity, we will just enter the username as a string for _user_profile_json_.  

If _override_ bin is set _true_, do not flag as fraudulent. We will use override true to insert our training data as well as allow the application to enter a genuine new payee.



## Record Data Model
While Aerospike limits bin names to 15 characters, shorter the better.

### Fraud Detection - Proposed Record Data Model:
* acct_id (Primary Key):
 *  ovr (override bin, boolean)
 *  lastamnt (last successful withdrawl_amount bin, integer, cents) 
 *  payee (payee bin, string)  
 *  bal (current_balance bin, integer, cents) 
 *  profile (user_profile_json bin, string, username) 
 
 ![fig_DM_2](./graphics/Fraud_DM_2.png)


#### New  _acct_id_ record creation

**Exercise** Write code to insert one record in namespace: **test**, setname: **accts**, key = **1** integer value (int acct_id=1). 
Add the bins described above with ovr=true, lastamnt=-10000, payee="SELF", bal=10000. 

(We will update new acct creation later with _automatic calculation of balance_.)

In [9]:
# New acct_id record creation
# Lets create a test record with this data structure

iAcct_id = 1 #Acct_id = 1 to 1000.
bOvr = True  # boolean - override, to be used later
iWdAmount = -10000 # negative amount is a deposit
sPayee = "SELF"
iBal = 10000  # initial balance after the deposit

user = ("test", "accts", iAcct_id)  

bins = { "ovr":bOvr, "lastamnt":iWdAmount, "payee":sPayee, "bal":iBal, "profile":"user"+str(iAcct_id) }

# Use Default wPolicy and default metadata

client.put(user,bins);



In [10]:
# Check 
(key,meta,bins) = client.get(user)
print(bins);

{'ovr': True, 'lastamnt': -10000, 'payee': 'SELF', 'bal': 10000, 'profile': 'user1'}


#### Update  _acct_id_ record with incoming transaction

Update record with a withdrawl transaction for `$80.00` or integer value 8000 cents. Recall we only have `$100.00`or 10000 cents. We should be left with 2000 cents. 

**Incoming Transaction:**
 *  acct_id:  _**override**_, _**withdrawl_amount**_, _**payee**_ 

How can we do this atomically? --> Use Write Expressions.


## Using OperationExpression write()



## Record Transaction Considerations

In the code below, note my own local aliases (exp, oh, opexp) implemented in the import statement. 

In [11]:
from aerospike_helpers import expressions as exp  #To define expressions
from aerospike_helpers.operations import operations as oh  #operation helper
from aerospike_helpers.operations import expression_operations as opexp #Expression Operations (read, write)
from aerospike import exception as ex

iAcct_id = 1  #Acct_id = 1 to 1000.
bOvr = False  # Later, we will use the override boolean value for decisioning.
iWdAmount = 8000
sPayee = "VISA"

user = ("test", "accts", iAcct_id); 

balExp = exp.Sub(exp.IntBin("bal"), iWdAmount).compile()

# wPolicy does not use any filter expression (yet!)
# Update withdrawl amount in amnt bin
# Expression Operation write for bal bin

ops = [
    oh.write("ovr",bOvr),
    oh.write("lastamnt", iWdAmount),     
    oh.write("payee", sPayee),               
    opexp.expression_write("bal", balExp, aerospike.EXP_WRITE_DEFAULT)
]
retVal = client.operate(user, ops, policy=wPolicy) 


In [12]:
# Check 
(key,meta,bins) = client.get(user)
print(bins);

{'ovr': False, 'lastamnt': 8000, 'payee': 'VISA', 'bal': 2000, 'profile': 'user1'}


#### (Recap: Record Data Model)

 ![fig_DM_2](./graphics/Fraud_DM_2.png)


### Simple balance check
If **withdrawl_amount**  _is greater than_ **current_balance**, reject the transaction. 

Lets modify our code to add this Filter Expression condition.

In [13]:
iAcct_id = 1  #Acct_id = 1 to 1000.
bOvr = False
iWdAmount = 3000
sPayee = "Cash"

user = ("test", "accts", iAcct_id) 

# Filter expression is part of the policy object, supplied when calling 'operate'
filterExp = exp.Or(
    exp.GE(exp.IntBin("bal"),iWdAmount), 
    bOvr
    ).compile()

wPolicy = {"expressions":filterExp}

# Will generate FilteredOut Exception when filterExp = false


balExp = exp.Sub(exp.IntBin("bal"), iWdAmount).compile()

ops = [
    oh.write("ovr",bOvr),
    oh.write("lastamnt", iWdAmount),     
    oh.write("payee", sPayee),               
    opexp.expression_write("bal", balExp, aerospike.EXP_WRITE_DEFAULT)
]

# wPolicy now has a filter expression added
# Update withdrawl amount in amnt bin
# Expression Operation write for bal bin

try:
    retVal = client.operate(user, ops, policy=wPolicy) 
except ex.FilteredOut as e:
    print("Insufficient Balance. Transaction Rejected.")
    #None
except:
    print("Something else went wrong")    
else:
    print("Transaction suceeded.")
finally:
    (key,meta,bins) = client.get(user)
    print(bins);

Insufficient Balance. Transaction Rejected.
{'ovr': False, 'lastamnt': 8000, 'payee': 'VISA', 'bal': 2000, 'profile': 'user1'}


### Application initiates first transaction to create the account.

To create the account in this first transaction in the account, the transaction is expected to be made with **override** _true_. _withdrawl_amount_ should be a negative number (deposit), starting _current_balance_ on new account creation will be zero, after the deposit, it will _(0 - (-amount)) = amount deposited_.  Add user profile (user name string). For payee name, use SELF.

If Bin "bal" exists, (new balance = old balance - withdrawl_amount), else (0 - withdrawl_amount).

If profile bin exists, do not update it otherwise add profile string in profile bin. (Use Expression Write Flags)

We are still working with this record data model:

![fig_DM_2](./graphics/Fraud_DM_2.png)


In [14]:
iAcct_id = 2 #Acct_id = 1 to 1000.
bOvr = True  # boolean - override
iWdAmount = -5000 # negative amount is a deposit
sPayee = "SELF"
sProfile = "user"+str(iAcct_id)

user = ("test", "accts", iAcct_id) 

# Filter expression - same as before
filterExp = exp.Or(
    exp.GE(exp.IntBin("bal"),iWdAmount), 
    bOvr
    ).compile()

wPolicy = {"expressions":filterExp}

# Will generate FilteredOut Exception when filterExp = false

# Expression for calculating balance
balExp = exp.Cond(exp.BinExists("bal"), exp.Sub(exp.IntBin("bal"), iWdAmount),
         exp.Sub(0, iWdAmount)).compile()
 
# Show use of EXP WRITE FLAGS    
# We will use Cond expression to create and expression from a string value
# We want to ensure that the profile only gets written once
profileExp = exp.Cond(True, sProfile, "None").compile()
 
ops = [
    oh.write("ovr",bOvr),
    oh.write("lastamnt", iWdAmount),     
    oh.write("payee", sPayee),
    opexp.expression_write("profile", profileExp, aerospike.EXP_WRITE_CREATE_ONLY|aerospike.EXP_WRITE_POLICY_NO_FAIL), 
    #Write to the profile bin on creation only. 'Fail' silently otherwise
    opexp.expression_write("bal", balExp, aerospike.EXP_WRITE_DEFAULT)  # Balance update
]    
 
# wPolicy has a filter expression
# Update withdrawl amount in amnt bin
# Expression Operation write for bal and profile bin with different WRITE FLAGS

try:
    retVal = client.operate(user, ops, policy=wPolicy) 
except ex.FilteredOut as e:
    print("Insufficient Balance. Transaction Rejected.")
    #None
except:
    print("Something else went wrong")    
else:
    print("Transaction suceeded.")
finally:
    (key,meta,bins) = client.get(user)
    print(bins);   


Transaction suceeded.
{'ovr': True, 'lastamnt': -5000, 'payee': 'SELF', 'profile': 'user2', 'bal': 5000}


### If it is a Deposit (Negative Withdrawl Amount):
* Skip Fraud Check
* Update **current_balance** and **withdrawl_amount**, and return.

Previous code should work with **override** = _false_.

Try and check with acct_id = 2. Add -3000 amount

**User profile should not update in this transaction.**

In [15]:
iAcct_id = 2  #Acct_id = 1 to 1000.
bOvr = False
iWdAmount = -3000
sPayee = "SELF"
sProfile = "do_not_update_user"+str(iAcct_id)  #Check we are not updating the profile string now.

user = ("test", "accts", iAcct_id) 

# Filter expression - same as before
filterExp = exp.Or(
    exp.GE(exp.IntBin("bal"),iWdAmount), 
    bOvr
    ).compile()

wPolicy = {"expressions":filterExp}
# Will generate FilteredOut Exception when filterExp = false

# Expression for calculating balance
balExp = exp.Cond(exp.BinExists("bal"), exp.Sub(exp.IntBin("bal"), iWdAmount),
         exp.Sub(0, iWdAmount)).compile()

# We want to ensure that the profile only gets written once
profileExp = exp.Cond(True, sProfile, "None").compile()
  
    
ops = [
    oh.write("ovr",bOvr),
    oh.write("lastamnt", iWdAmount),     
    oh.write("payee", sPayee),
    opexp.expression_write("profile", profileExp, aerospike.EXP_WRITE_CREATE_ONLY|aerospike.EXP_WRITE_POLICY_NO_FAIL), 
    #Write to the profile bin on creation only. 'Fail' silently otherwise
    opexp.expression_write("bal", balExp, aerospike.EXP_WRITE_DEFAULT)  # Balance update
] 

# wPolicy has a filter expression
# Update withdrawl amount in amnt bin
# Expression Operation write for bal and profile bin with different WRITE FLAGS

try:
    retVal = client.operate(user, ops, policy=wPolicy) 
except ex.FilteredOut as e:
    print("Insufficient Balance. Transaction Rejected.")
    #None
except:
    print("Something else went wrong")    
else:
    print("Transaction suceeded.")
finally:
    (key,meta,bins) = client.get(user)
    print(bins);  

Transaction suceeded.
{'ovr': False, 'lastamnt': -3000, 'payee': 'SELF', 'profile': 'user2', 'bal': 8000}


**Check**
- Should have an 8000 balance (we added 3000 to the user iAcct_id=2's account)
- **profile:** should still read _user2_ (should not update to _"do_not_update_user2"_ )

==> Demonstrates use of Expression Write flags in Expressson Operation write()


### Should it be flagged as "Potential_Fraud"?
We will implement a simple **Go/NoGo** Model - we are not computing a fraud score in our hands-on exercise today.
For each **_acct_id_** : 



#### Maintain in acct_id record:
When a new account is created, add fraud detection criteria:
* Fraud criteria (30 percent over average threshold, thresh = 30), can be tuned in future. (Do once when acct_id record is created.)

Upon incoming withdrawl transaction, atomically maintain:
* List of unique payees.
* Maintain average withdrawl amount for each payee.  
_(Maintain sum of all withdrawls, count of withdrawl. avg = sum/count)_ 
* **current_balance** if transaction is not rejected or flagged as fraudulent

We will internally maintain the following additional bins. Data will be updated with every incoming transaction, atomically, by using Expressions.

![fig_DM_3](./graphics/Fraud_DM_3.png)





In [16]:
from aerospike_helpers import cdt_ctx
from aerospike_helpers.expressions import resources as er
from aerospike_helpers.operations import map_operations as mapop

iAcct_id = 3  #Acct_id = 1 to 1000.
bOvr = True  #<-- Note: we set Overrride to false  (wrong)
iWdAmount = -2000
sPayee = "SELF";
sProfile = "user"+str(iAcct_id)  
iThreshold = 30

user = ("test", "accts", iAcct_id) 

wPolicy = {}  #Write Policy is dictionary. We will populate it as needed.

if(bOvr):
    wPolicy["exists"] = aerospike.POLICY_EXISTS_CREATE  #Create only if it does not exist, when creating acct_id record
else:
    wPolicy["exists"] = aerospike.POLICY_EXISTS_UPDATE  #Update only if it exists, acct_id record must exist.

#User 3 account is not created yet. If we create with bOvr=false, we should get an error.


#Note the use of Cond() for if then kind of logic. 
#In this case, 
# Check if bal bin exists. If yes, check bal>=iWdAmount, if not return False
# Unless (OR with bOvr), if bOvr is True, proceed.

filterExp = exp.Or(
    exp.Cond(exp.BinExists("bal"), exp.GE(exp.IntBin("bal"),iWdAmount), False), 
    bOvr
    ).compile()
wPolicy["expressions"] = filterExp

# Balance computation, same as before.
balExp = exp.Cond(exp.BinExists("bal"), exp.Sub(exp.IntBin("bal"),iWdAmount),
                  exp.Sub(0, iWdAmount) ).compile()

#Create expressions out of values for using in Expression Operation write()
profileExp = exp.Cond(True, sProfile, "None").compile()  #string data, default "None" 
thresholdExp = exp.Cond(True, iThreshold, 0).compile()   #integer data, default 0

mPolicy = {
    'map_order': aerospike.MAP_KEY_ORDERED,
    'map_write_flags': aerospike.MAP_WRITE_FLAGS_DEFAULT
}

#If MapKey sPayee exists, add to sum and count, otherwise create sum and count entries for map key = sPayee
#Value returned by this expression will be written to sPayeee{sum:expValue}

payee_ctx = [cdt_ctx.cdt_ctx_map_key(sPayee)]  #specify ctx as a LIST type 

payeeSumExp = exp.Cond(   #IF ....
        # The number of K:V pair corresponding to key sPayee in the mapPayees bin
        exp.Eq( exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ), 
                # is equal to zero       
                0), 
                # then the sum of payments to sPayee is equal to the current payment amount
                iWdAmount,
        # else if this count
        exp.Eq( exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ), 
               # is equal to one
               1), 
               # then get the currently stored sum of all payments for this payee - use payee_ctx
               exp.Add(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, 
                              "sum", exp.MapBin("mapPayees") 
                              ), 
                          # and add to it the payment amount        
                          iWdAmount),
                          0 #default
            ) #Do not compile(), this is just Exp construct, not an Expression yet.
    
# This expression saves the result of the payeeSumExp calculation
savePayeeSumExp = exp.MapPut(payee_ctx, mPolicy, "sum", payeeSumExp, 
                             # for the mapPayees bin 
                             exp.MapBin("mapPayees") ).compile()
                             

#Value returned by payeeCountExp will be written to sPayeee{count:expValue}
#The structure is very similar to sumCountExp
#Except that we set the count to 1 if the payee has not been seen before
#And increment the count if it has
payeeCountExp = exp.Cond(   #IF .....
        exp.Eq( 
            # The number of K:V pair corresponding to key sPayee in the mapPayees bin
            exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ),
            # is equal to zero 
            0), 
            # then return 1
            1,
            # else if this count 
        exp.Eq( 
            exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ),
            # is equal to one         
            1), 
            # then get the currently stored count of all payments for this payee 
            exp.Add(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, 
                              "count", exp.MapBin("mapPayees")),                                             
                    # and add one to it
                    1),
            # Finally if number of K:V pairs is > 1 ( can't happen) return 0
            0)

#Similarly this expression saves the result of the payeeCountExp calculation

savePayeeCountExp = exp.MapPut(payee_ctx, mPolicy, "count", payeeCountExp, 
                             # for the mapPayees bin 
                             exp.MapBin("mapPayees") ).compile()

# Map to initialize sum and count to 0
initPayeeMap = {"sum":0, "count":0}

# The map policy we use will make sure the initalization map is only saved if it doesn't exist
mPolicyInit = {
    'map_order': aerospike.MAP_KEY_ORDERED,
    'map_write_flags': aerospike.MAP_WRITE_FLAGS_CREATE_ONLY|
                       aerospike.MAP_WRITE_FLAGS_NO_FAIL|
                       aerospike.MAP_WRITE_FLAGS_PARTIAL
}

ops = [
    oh.write("ovr",bOvr),
    oh.write("lastamnt", iWdAmount),     
    oh.write("payee", sPayee),
    opexp.expression_write("bal", balExp, aerospike.EXP_WRITE_DEFAULT),  # Balance update
    #Write to the profile and threshold bin on creation only. 'Skip' silently otherwise
    opexp.expression_write("profile",  profileExp, 
                           aerospike.EXP_WRITE_CREATE_ONLY|aerospike.EXP_WRITE_POLICY_NO_FAIL), 
    opexp.expression_write("thresh", thresholdExp, 
                           aerospike.EXP_WRITE_CREATE_ONLY|aerospike.EXP_WRITE_POLICY_NO_FAIL),
    #Initialize payee map if required
    mapop.map_put("mapPayees", sPayee, initPayeeMap, mPolicyInit),
    opexp.expression_write("mapPayees", savePayeeSumExp, aerospike.EXP_WRITE_EVAL_NO_FAIL),  # payee: sum update
    opexp.expression_write("mapPayees", savePayeeCountExp, aerospike.EXP_WRITE_EVAL_NO_FAIL) # payee: count update
]

# wPolicy has a filter expression
# Update withdrawl amount in amnt bin
# Expression Operation write for bal and profile bin with different WRITE FLAGS

try:
    retVal = client.operate(user, ops, policy=wPolicy) 
except ex.FilteredOut as e:
    print("Insufficient Balance. Transaction Rejected.")
    #None
except Exception as err:
    print(type(err))
    print(err)
    print("Something else went wrong")    
else:
    print("Transaction suceeded.")
finally:
    (key,meta,bins) = client.get(user)
    print(bins);  

Transaction suceeded.
{'ovr': True, 'lastamnt': -2000, 'payee': 'SELF', 'bal': 2000, 'profile': 'user3', 'thresh': 30, 'mapPayees': {'SELF': {'sum': -2000, 'count': 1}}}


In [17]:
#Check 
user = ("test", "accts", 3) 
_,_,bins = client.get(user)
print( bins )
#client.remove(user)  #use during code development

{'ovr': True, 'lastamnt': -2000, 'payee': 'SELF', 'bal': 2000, 'profile': 'user3', 'thresh': 30, 'mapPayees': {'SELF': {'sum': -2000, 'count': 1}}}


#### Update Record Data Model for Fraud Detection related bins
* payeelist Map data type with key=payee name, value = [cumulative_withdawls, count] to compute average.  Estimate record size vs max number of payees. 


![fig_DM_3](./graphics/Fraud_DM_3.png)


* Decide record size to use 128K / 256K / 512K / 1M / 2M / 4M / 8M? 1M or below is preferred.
 *  **Sizing Estimate:**
 *  Use device storage sizing guideline to make an estimate using an average payee name string length.
 *  Should we hash the payee name to known number of bits to get consistent sizing? 
  *   CDTs allow byte arrays to be map keys.
  *   Stay with string payee name map key for today's exercise.
* Fraud threshold percentage bin: _thresh_ = 30. 


#### Flag as Potential_Fraud if override is false and:
* If _withdrawl_amount_ is greater than x (e.g.30)% of average. 
* Skip fraud check on first 5 transactions of a payee to build the average.
* Don't apply fraud checkn on Credit amounts (negative withdrawl)

* **If flagged as Potential_Fraud:** 
 *  do not add payee to unique list
 *  do not update payee average withdrawl amount.
 *  do not update transaction withdrawl_amount or current_balance.
 

In [18]:
iAcct_id = 3  #Acct_id = 1 to 1000.
bOvr = False 
iWdAmount = 1200
sPayee = "VISA";
sProfile = "user"+str(iAcct_id)  
iThreshold = 30

user = ("test", "accts", iAcct_id) 

wPolicy = {}  #Write Policy is dictionary. We will populate it as needed.

if(bOvr):
    wPolicy["exists"] = aerospike.POLICY_EXISTS_CREATE  #Create only if it does not exist, when creating acct_id record
else:
    wPolicy["exists"] = aerospike.POLICY_EXISTS_UPDATE  #Update only if it exists, acct_id record must exist.

#User 3 account is not created yet. If we create with bOvr=false, we should get an error.


#Note the use of Cond() for if then kind of logic. 
#In this case, 
# Check if bal bin exists. If yes, check bal>=iWdAmount, if not return False
# Unless (OR with bOvr), if bOvr is True, proceed.


payee_ctx = [cdt_ctx.cdt_ctx_map_key(sPayee)]  #specify ctx as a LIST type

# Take the existing filter expression and add to it a fraud detection filter
filterExp = exp.And(
              exp.Or(
               exp.Cond(exp.BinExists("bal"), exp.GE(exp.IntBin("bal"),iWdAmount), False),
               #Note: This filter will be ignored if the record does not exist
               #Hence, we use record Exists policy per above based on bOvr value.
               bOvr),
      
            #Lets now add fraud detection filter- 
            #Withdrawl amount should be within 30% of average withdrawl amount.    
      
             exp.Cond(exp.LT(iWdAmount,0), True,
              #First if withdrawl amount is negative (<0), return true  
              #then, if mapPayees bin does not exist, return true, no transaction in yet.
              exp.Not(exp.BinExists("mapPayees")), True,                
              #then, if mapPayees does not have entry for sPayee, return true, no transaction in yet
              exp.Eq(exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ), 0), True,                  
              #Then, if count <5, (we allow first 5 entries before computing fraud detection), return True
              exp.LT(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, "count", 
                                      exp.MapBin("mapPayees")), 5), True,                     
              #check withdrawl amount <= (100+threshold)*(sum/count)/100
              exp.LE(iWdAmount,
                exp.Div(
                  exp.Mul(exp.Add(100,exp.IntBin("thresh")), 
                    exp.Div(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, "sum", 
                            exp.MapBin("mapPayees")),
                            exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, "count", 
                            exp.MapBin("mapPayees")))                
                        ),
                        100) 
                     ),  True,  # and if so, return true             
              False #else return false
             )  #Cond
           ).compile() #And

wPolicy["expressions"] = filterExp


# Balance computation, same as before.
balExp = exp.Cond(exp.BinExists("bal"), exp.Sub(exp.IntBin("bal"),iWdAmount),
                  exp.Sub(0, iWdAmount) ).compile()

#Create expressions out of values for using in Expression Operation write()
profileExp = exp.Cond(True, sProfile, "None").compile()  #string data, default "None" 
thresholdExp = exp.Cond(True, iThreshold, 0).compile()   #integer data, default 0

mPolicy = {
    'map_order': aerospike.MAP_KEY_ORDERED,
    'map_write_flags': aerospike.MAP_WRITE_FLAGS_DEFAULT
}

#If MapKey sPayee exists, add to sum and count, otherwise create sum and count entries for map key = sPayee
#Value returned by this expression will be written to sPayeee{sum:expValue}

payeeSumExp = exp.Cond(   #IF ....
        # The number of K:V pair corresponding to key sPayee in the mapPayees bin
        exp.Eq( exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ), 
                # is equal to zero       
                0), 
                # then the sum of payments to sPayee is equal to the current payment amount
                iWdAmount,
        # else if this count
        exp.Eq( exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ), 
               # is equal to one
               1), 
               # then get the currently stored sum of all payments for this payee - use payee_ctx
               exp.Add(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, 
                              "sum", exp.MapBin("mapPayees") 
                              ), 
                          # and add to it the payment amount        
                          iWdAmount),
                          0 #default
            ) #Do not compile(), this is just Exp construct, not an Expression yet.
    
# This expression saves the result of the payeeSumExp calculation
savePayeeSumExp = exp.MapPut(payee_ctx, mPolicy, "sum", payeeSumExp, 
                             # for the mapPayees bin 
                             exp.MapBin("mapPayees") ).compile()
                             

#Value returned by payeeCountExp will be written to sPayeee{count:expValue}
#The structure is very similar to sumCountExp
#Except that we set the count to 1 if the payee has not been seen before
#And increment the count if it has
payeeCountExp = exp.Cond(   #IF .....
        exp.Eq( 
            # The number of K:V pair corresponding to key sPayee in the mapPayees bin
            exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ),
            # is equal to zero 
            0), 
            # then return 1
            1,
            # else if this count 
        exp.Eq( 
            exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ),
            # is equal to one         
            1), 
            # then get the currently stored count of all payments for this payee 
            exp.Add(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, 
                              "count", exp.MapBin("mapPayees")),                                             
                    # and add one to it
                    1),
            # Finally if number of K:V pairs is > 1 ( can't happen) return 0
            0)

#Similarly this expression saves the result of the payeeCountExp calculation

savePayeeCountExp = exp.MapPut(payee_ctx, mPolicy, "count", payeeCountExp, 
                             # for the mapPayees bin 
                             exp.MapBin("mapPayees") ).compile()

# Map to initialize sum and count to 0
initPayeeMap = {"sum":0, "count":0}

# The map policy we use will make sure the initalization map is only saved if it doesn't exist
mPolicyInit = {
    'map_order': aerospike.MAP_KEY_ORDERED,
    'map_write_flags': aerospike.MAP_WRITE_FLAGS_CREATE_ONLY|
                       aerospike.MAP_WRITE_FLAGS_NO_FAIL|
                       aerospike.MAP_WRITE_FLAGS_PARTIAL
}

ops = [
    oh.write("ovr",bOvr),
    oh.write("lastamnt", iWdAmount),     
    oh.write("payee", sPayee),
    opexp.expression_write("bal", balExp, aerospike.EXP_WRITE_DEFAULT),  # Balance update
    #Write to the profile and threshold bin on creation only. 'Skip' silently otherwise
    opexp.expression_write("profile",  profileExp, 
                           aerospike.EXP_WRITE_CREATE_ONLY|aerospike.EXP_WRITE_POLICY_NO_FAIL), 
    opexp.expression_write("thresh", thresholdExp, 
                           aerospike.EXP_WRITE_CREATE_ONLY|aerospike.EXP_WRITE_POLICY_NO_FAIL),
    #Initialize payee map if required
    mapop.map_put("mapPayees", sPayee, initPayeeMap, mPolicyInit),
    opexp.expression_write("mapPayees", savePayeeSumExp, aerospike.EXP_WRITE_EVAL_NO_FAIL),  # payee: sum update
    opexp.expression_write("mapPayees", savePayeeCountExp, aerospike.EXP_WRITE_EVAL_NO_FAIL) # payee: count update
]

# wPolicy has a filter expression
# Update withdrawl amount in amnt bin
# Expression Operation write for bal and profile bin with different WRITE FLAGS

try:
    retVal = client.operate(user, ops, policy=wPolicy) 
except ex.FilteredOut as e:
    print("Insufficient Balance or Fraud Threshold Exceeded. Transaction Rejected.")
    #None
except Exception as err:
    print(type(err))
    print(err)
    print("Something else went wrong")    
else:
    print("Transaction suceeded.")
finally:
    (key,meta,bins) = client.get(user)
    print(bins);  


Transaction suceeded.
{'ovr': False, 'lastamnt': 1200, 'payee': 'VISA', 'bal': 800, 'profile': 'user3', 'thresh': 30, 'mapPayees': {'VISA': {'sum': 1200, 'count': 1}, 'SELF': {'sum': -2000, 'count': 1}}}


## Using Python classes and methods
 
 Lets see with a simple example how we can use Java classes and methods in Jupyter Notebook. We can clean up our implementation as a class method where we can pass the transaction attributes as an argument.

### Defing a Class
 

In [19]:
#CLASS Cell
class myTest:
    def __init__(self):
        None
    def foo (self, val ): 
        print("Calling myTest:foo() to print integer: ",val);
  

### Using a Class Method
 

In [20]:
myobj = myTest()
myobj.foo(5)

Calling myTest:foo() to print integer:  5


## "Classify" our Model
 Let create a class _detectFraud_ with a method transact() that takes in a transaction with all its attributes.

In [51]:
#Add the imports. We already have them in our interactive kernel at this point.
#Classify 
class DetectFraud:
    def __init__(self):
        None
    def transact(self, 
    iAcct_id,
    bOvr,
    iWdAmount,
    sPayee,
    iThreshold):
 
        sProfile = "user"+str(iAcct_id)  
        user = ("test", "accts", iAcct_id) 
        
        wPolicy = {}  #Write Policy is dictionary. We will populate it as needed.

        if(bOvr and (iWdAmount<0)):
            wPolicy["exists"] = aerospike.POLICY_EXISTS_CREATE  #Create only if it does not exist, when creating acct_id record
        else:
            wPolicy["exists"] = aerospike.POLICY_EXISTS_UPDATE  #Update only if it exists, acct_id record must exist.

        #User 3 account is not created yet. If we create with bOvr=false, we should get an error.


        #Note the use of Cond() for if then kind of logic. 
        #In this case, 
        # Check if bal bin exists. If yes, check bal>=iWdAmount, if not return False
        # Unless (OR with bOvr), if bOvr is True, proceed.

        payee_ctx = [cdt_ctx.cdt_ctx_map_key(sPayee)]  #specify ctx as a LIST type

        # Take the existing filter expression and add to it a fraud detection filter               
        filterExp = exp.And(
            exp.Or(
               exp.Cond(exp.BinExists("bal"), exp.GE(exp.IntBin("bal"),iWdAmount), False),
               #Note: This filter will be ignored if the record does not exist
               #Hence, we use record Exists policy per above based on bOvr value.
               bOvr),
      
            #Lets now add fraud detection filter- 
            #Withdrawl amount should be within 30% of average withdrawl amount.    
      
            exp.Cond(bOvr, True, #if override is true, skip fraud check.
              exp.LT(iWdAmount,0), True, #If withdrawl amount is negative (<0), return true  
              #then, if mapPayees bin does not exist, return true, no transaction in yet.
              exp.Not(exp.BinExists("mapPayees")), True,                
              #then, if mapPayees does not have entry for sPayee, return true, no transaction in yet
              exp.Eq(exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ), 0), True,                  
              #Then, if count <5, (we allow first 5 entries before computing fraud detection), return True
              exp.LT(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, "count", 
                                      exp.MapBin("mapPayees")), 5), True,                     
              #check withdrawl amount <= (100+threshold)*(sum/count)/100
              exp.LE(iWdAmount,
                exp.Div(
                  exp.Mul(exp.Add(100,exp.IntBin("thresh")), 
                    exp.Div(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, "sum", 
                            exp.MapBin("mapPayees")),
                            exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, "count", 
                            exp.MapBin("mapPayees")))                
                        ),
                        100) 
                     ),  True,  # and if so, return true             
              False #else return false
             )  #Cond
           ).compile() #And

        wPolicy["expressions"] = filterExp

        # Balance computation, same as before.
        balExp = exp.Cond(exp.BinExists("bal"), exp.Sub(exp.IntBin("bal"),iWdAmount),
                  exp.Sub(0, iWdAmount) ).compile()

        #Create expressions out of values for using in Expression Operation write()
        profileExp = exp.Cond(True, sProfile, "None").compile()  #string data, default "None" 
        thresholdExp = exp.Cond(True, iThreshold, 0).compile()   #integer data, default 0

        mPolicy = {
        'map_order': aerospike.MAP_KEY_ORDERED,
        'map_write_flags': aerospike.MAP_WRITE_FLAGS_DEFAULT
        }

        #If MapKey sPayee exists, add to sum and count, otherwise create sum and count entries for map key = sPayee
        #Value returned by this expression will be written to sPayeee{sum:expValue}

        payeeSumExp = exp.Cond(   #IF ....
        # The number of K:V pair corresponding to key sPayee in the mapPayees bin
        exp.Eq( exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ), 
                # is equal to zero       
                0), 
                # then the sum of payments to sPayee is equal to the current payment amount
                iWdAmount,
        # else if this count
        exp.Eq( exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ), 
               # is equal to one
               1), 
               # then get the currently stored sum of all payments for this payee - use payee_ctx
               exp.Add(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, 
                              "sum", exp.MapBin("mapPayees") 
                              ), 
                          # and add to it the payment amount        
                          iWdAmount),
                          0 #default
            ) #Do not compile(), this is just Exp construct, not an Expression yet.
    
        # This expression saves the result of the payeeSumExp calculation
        savePayeeSumExp = exp.MapPut(payee_ctx, mPolicy, "sum", payeeSumExp, 
                             # for the mapPayees bin 
                             exp.MapBin("mapPayees") ).compile()
                             

        #Value returned by payeeCountExp will be written to sPayeee{count:expValue}
        #The structure is very similar to sumCountExp
        #Except that we set the count to 1 if the payee has not been seen before
        #And increment the count if it has
        payeeCountExp = exp.Cond(   #IF .....
          exp.Eq( 
            # The number of K:V pair corresponding to key sPayee in the mapPayees bin
            exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ),
            # is equal to zero 
            0), 
            # then return 1
            1,
            # else if this count 
          exp.Eq( 
            exp.MapGetByKey(None, aerospike.MAP_RETURN_COUNT, er.ResultType.INTEGER, sPayee, 
                                      exp.MapBin("mapPayees") ),
            # is equal to one         
            1), 
            # then get the currently stored count of all payments for this payee 
            exp.Add(exp.MapGetByKey(payee_ctx, aerospike.MAP_RETURN_VALUE, er.ResultType.INTEGER, 
                              "count", exp.MapBin("mapPayees")),                                             
                    # and add one to it
                    1),
            # Finally if number of K:V pairs is > 1 ( can't happen) return 0
            0)

        #Similarly this expression saves the result of the payeeCountExp calculation

        savePayeeCountExp = exp.MapPut(payee_ctx, mPolicy, "count", payeeCountExp, 
                             # for the mapPayees bin 
                             exp.MapBin("mapPayees") ).compile()

        # Map to initialize sum and count to 0
        initPayeeMap = {"sum":0, "count":0}

        # The map policy we use will make sure the initalization map is only saved if it doesn't exist
        mPolicyInit = {
        'map_order': aerospike.MAP_KEY_ORDERED,
        'map_write_flags': aerospike.MAP_WRITE_FLAGS_CREATE_ONLY|
                       aerospike.MAP_WRITE_FLAGS_NO_FAIL|
                       aerospike.MAP_WRITE_FLAGS_PARTIAL
        }

        ops = [
          oh.write("ovr",bOvr),
          oh.write("lastamnt", iWdAmount),     
          oh.write("payee", sPayee),
          opexp.expression_write("bal", balExp, aerospike.EXP_WRITE_DEFAULT),  # Balance update
          #Write to the profile and threshold bin on creation only. 'Skip' silently otherwise
          opexp.expression_write("profile",  profileExp, 
                           aerospike.EXP_WRITE_CREATE_ONLY|aerospike.EXP_WRITE_POLICY_NO_FAIL), 
          opexp.expression_write("thresh", thresholdExp, 
                           aerospike.EXP_WRITE_CREATE_ONLY|aerospike.EXP_WRITE_POLICY_NO_FAIL),
          #Initialize payee map if required
          mapop.map_put("mapPayees", sPayee, initPayeeMap, mPolicyInit),
          opexp.expression_write("mapPayees", savePayeeSumExp, aerospike.EXP_WRITE_EVAL_NO_FAIL),  # payee: sum update
          opexp.expression_write("mapPayees", savePayeeCountExp, aerospike.EXP_WRITE_EVAL_NO_FAIL) # payee: count update
        ]

        # wPolicy has a filter expression
        # Update withdrawl amount in amnt bin
        # Expression Operation write for bal and profile bin with different WRITE FLAGS

        try:
            retVal = client.operate(user, ops, policy=wPolicy) 
        except ex.FilteredOut as e:
            print("Insufficient Balance or Fraud Threshold Exceeded. Transaction Rejected.")
            #None
        except Exception as err:
            print(type(err))
            print(err)
            print("Something else went wrong")    
        else:
            print("Transaction suceeded.")
            #None
        finally:
            #(key,meta,bins) = client.get(user)
            #print(bins);
            None


## Test our "Classified" Model
 

### Restart clean on Aerospike server
 

In [22]:
%%sh
aql 
truncate test
exit

Seed:         127.0.0.1
User:         None
Config File:  /etc/aerospike/astools.conf /home/training/.aerospike/astools.conf 
Aerospike Query Client
Version dev
Copyright 2012-2021 Aerospike. All rights reserved.
aql> truncate test
OK

aql> exit



C Client Version 6.0.0


In [23]:
myFraudDetector = DetectFraud()
#myFraudDetector.transact(1,False,1000,"VISA",30)
myFraudDetector.transact(1,True,-5000,"SELF",30)
myFraudDetector.transact(1,False,-10000,"SELF",30)
myFraudDetector.transact(1,False,1000,"VISA",30)
myFraudDetector.transact(1,False,1500,"Costco",30)
myFraudDetector.transact(1,False,500,"VISA",30)
myFraudDetector.transact(1,False,1000,"VISA",30)
myFraudDetector.transact(1,False,1500,"Costco",30)
myFraudDetector.transact(1,False,-5000,"SELF",30)
myFraudDetector.transact(1,False,1000,"VISA",30)
myFraudDetector.transact(1,False,2000,"Costco",30)
myFraudDetector.transact(1,False,1500,"VISA",30)
myFraudDetector.transact(1,False,1000,"VISA",30)
myFraudDetector.transact(1,False,3000,"VISA",30)
myFraudDetector.transact(1,False,1000,"VISA",30)

Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Transaction suceeded.
Insufficient Balance or Fraud Threshold Exceeded. Transaction Rejected.
Transaction suceeded.


In [24]:
#Check user 1
user = ("test", "accts", 1) 
_,_,bins = client.get(user)
print( bins )

{'ovr': False, 'lastamnt': 1000, 'payee': 'VISA', 'bal': 8000, 'profile': 'user1', 'thresh': 30, 'mapPayees': {'VISA': {'sum': 7000, 'count': 7}, 'SELF': {'sum': -20000, 'count': 3}, 'Costco': {'sum': 5000, 'count': 3}}}


### Try other transactions
 

### Clean-up Aerospike server

We will populate with a larger dataset.
 

In [46]:
%%sh
aql 
truncate test
exit

Seed:         127.0.0.1
User:         None
Config File:  /etc/aerospike/astools.conf /home/training/.aerospike/astools.conf 
Aerospike Query Client
Version dev
Copyright 2012-2021 Aerospike. All rights reserved.
aql> truncate test
OK

aql> exit



C Client Version 6.0.0


## Insert Generated Random Test Data
Let's create 1000 acct_ids, and then insert ~100K transactions for 1000 acct_ids [0001 thru 1000], average 10 transactions per acct_id per payee, each acct_id has ~10 unique payees. (Average 100 transactions per payee per acct_id).

1. Create acct_ids
2. Upload/insert transacitons. (Underlying code should build needed fraud detection bins' data with every transaction.)
3. Transactions include deposit transactions also.
4. All these inserts are done with override=true.

**Payee names in Test Data:** VISA, CitiMortgage, Costco, HOA, Joe_Landscaper, PacificElectric, CityWater, Jane_Helper, John_Doe, Cash. (14 characters max here.)


In [47]:
import random

class GenerateTestData:
    def __init__(self):
        None

    def generate(self):
        myFraudDetector = DetectFraud()

        #Create acct_id with initial balance of 10000
        for i in range(1, 1001):
            myFraudDetector.transact(i,True,-10000,"SELF",30)


        payee = ["VISA","CitiMortgage","Costco","HOA","Joe_Landscaper",
                 "PacificElectric","CityWater","Jane_Helper","John_Doe","Cash"]
        index = [0,1,2,3,4,5]
        for i in range(1, 1001):
            for j in range(10):
                for k in range(random.choice(index),10): #10 unique payees
                    myFraudDetector.transact(i,True,(200+(20*random.choice(index))),payee[k],30);
                    #Override is true - no fraud detection enabled.
     
                myFraudDetector.transact(i,False,-4000,"SELF",30);  #Replenish balance


In [48]:
#Execute
genData = GenerateTestData()
genData.generate()

## Test the Fraud Model
Insert a fraudulent transaction. Typically, it will be a payee withdrawl as "Cash" with a suspicious amount.


In [49]:
#Check any user
user = ("test", "accts", 5) 
(_,_,bins) = client.get(user) 
print(bins)

{'ovr': False, 'lastamnt': -4000, 'payee': 'SELF', 'bal': 31620, 'profile': 'user5', 'thresh': 30, 'mapPayees': {'VISA': {'sum': 260, 'count': 1}, 'John_Doe': {'sum': 2440, 'count': 10}, 'CityWater': {'sum': 2420, 'count': 10}, 'Cash': {'sum': 2420, 'count': 10}, 'Joe_Landscaper': {'sum': 1820, 'count': 7}, 'CitiMortgage': {'sum': 740, 'count': 3}, 'Costco': {'sum': 1600, 'count': 6}, 'HOA': {'sum': 1580, 'count': 6}, 'Jane_Helper': {'sum': 2560, 'count': 10}, 'PacificElectric': {'sum': 2540, 'count': 10}, 'SELF': {'sum': -50000, 'count': 11}}}


### Test for this user
Insert a fraudulent transaction. Typically, it will be a payee withdrawl as "Cash" with a suspicious amount.


In [55]:
myFraudDetector = DetectFraud()
myFraudDetector.transact(5,False,400,"Cash",30);
print("Override Fraud Detection and Retry:")
myFraudDetector.transact(5,True,400,"Cash",30);

Insufficient Balance or Fraud Threshold Exceeded. Transaction Rejected.
Override Fraud Detection and Retry:
Transaction suceeded.


In [56]:
#Check last transaction (if it succeeds)
user = ("test", "accts", 5) 
(_,_,bins) = client.get(user) 
print(bins)

{'ovr': True, 'lastamnt': 400, 'payee': 'Cash', 'bal': 30100, 'profile': 'user5', 'thresh': 30, 'mapPayees': {'VISA': {'sum': 260, 'count': 1}, 'John_Doe': {'sum': 2440, 'count': 10}, 'CityWater': {'sum': 2420, 'count': 10}, 'Cash': {'sum': 3940, 'count': 14}, 'Joe_Landscaper': {'sum': 1820, 'count': 7}, 'CitiMortgage': {'sum': 740, 'count': 3}, 'Costco': {'sum': 1600, 'count': 6}, 'HOA': {'sum': 1580, 'count': 6}, 'Jane_Helper': {'sum': 2560, 'count': 10}, 'PacificElectric': {'sum': 2540, 'count': 10}, 'SELF': {'sum': -50000, 'count': 11}}}


## Model Tuning
 Have a way to update the fraud detection criteria, in our example, value in the threshold percentage bin (_thresh_). 
 Scan all records (we have 1000) and update _threshold percentage_ value in each record.
 
 
![fig_DM_3](./graphics/Fraud_DM_3.png)



In [31]:
iThreshold = 65

thresholdExp = exp.Cond(True, iThreshold, 0).compile()   #integer data, default 0

ops = [ opexp.expression_write("thresh", thresholdExp, aerospike.EXP_WRITE_DEFAULT) ]

scan = client.scan('test', 'accts')

scan.add_ops(ops)

id = scan.execute_background()

## Retest
Hop back to previous cells, for a particular user, check new threshold, test. Calculate fraud threshold from record data, insert a fraudulent transaction and check. Insert a good transaction and check.

# Epilogue
We have assumed an extremely simple example to deliver a hands-on experience in the limited amount of time on our hands. The goal was to demonstrate some of the capabilities and features available with Aerospike. While experts in fraud detection will develop much more sophisticated models, we hope they find these techinques helpful.

# But wait, there is more :-)
We also have the Aerospike Monitoring Stack running on our single AWS Instance.

![fig_setup_2](./graphics/fraud_det_iPython_setup.png)

Lets take a quick look at the Grafana Dashboard in the browser. 

Go to your instance's  Public IP address and port 8000. 

### http://your_public_ip_addr:8000 

### Login Id: admin

### Password: admin

### SKIP Changing the password.

![fig_gf_1](./graphics/grafana_login.png)

### Load _Cluster Overview_ and _Namespace View_ Dashboards from _Aerospike_ folder


![fig_gf_2](./graphics/grafana_dashboard_load.png)

### Cluster Overview Dashboard

Shows information about the Aerospike Cluster

![fig_gf_3](./graphics/cluster_view.png)

### Namespace View Dashboard 

Should show 1K Master Objects

![fig_gf_4](./graphics/namespace_view.png)





# But wait, I can't Deploy Everything on a Single Machine in Practice


# How can I quickly Deploy something like this ...


![fig_qs_1](./graphics/aws_quickstart.png)


## Yes you can - with  AWS Quickstart  - Lets Explore that next.


